In [14]:
import pandas as pd

data = 'D:/Users/daniel.godden/Data/filtered_subnat_data_MATCHED.csv'

df_0 = pd.read_csv('D:/Users/daniel.godden/Data/lookup_data/bua_lad_lookup.csv')
df_1 = pd.read_csv('D:/Users/daniel.godden/Data/lookup_data/ca_lad_lookup.csv')
df_2 = pd.read_csv('D:/Users/daniel.godden/Data/lookup_data/freeports_lad_lookup.csv')
df_3 = pd.read_csv('D:/Users/daniel.godden/Data/lookup_data/lep_lad_lookup.csv')
df_4 = pd.read_csv('D:/Users/daniel.godden/Data/lookup_data/lad_county_lookup.csv')

lookup_dict = {'BUA':df_0,'CAUTH':df_1,'freeport':df_2, 'LEP':df_3,'LAD':df_4}

# import data and create new columns
df0 = pd.read_csv(data, low_memory=False)
df0['final_lad_code'] = 'NaN'
df0['final_lad_name'] = 'NaN'
df0['Derived_Amount'] = 'NaN'

# import fund_knowledge file and match projects from data with fund_knowledge
df1 = pd.read_csv('D:/Users/daniel.godden/Data/fund_knowledge.csv')
df0 = pd.merge(df0,df1.drop_duplicates(), on=['Segment_Name','Subsegment_Name'], how='left')

# if Area_Type is 'LAD' level, populate new columns with original
df0.loc[df0['Area_Type']=='LAD', 'Derived_Amount'] = df0.loc[df0['Area_Type']== 'LAD', 'Amount']
df0.loc[df0['Area_Type']=='LAD', 'final_lad_code'] = df0.loc[df0['Area_Type']== 'LAD', 'Local_Authority_Code']
df0.loc[df0['Area_Type']=='LAD', 'final_lad_name'] = df0.loc[df0['Area_Type']== 'LAD', 'Local_Authority_Name']

# LEP
x ='LEP'

df2 = lookup_dict[x]
merge = df0.loc[df0['Area_Type']== x]
merge = merge[['Segment_Name','Local_Authority_Code', 'Amount']]
merge['LAD22CD'] = merge['Local_Authority_Code']
merge.drop(['Local_Authority_Code'], axis=1, inplace=True)
merge = pd.merge(df2, merge.drop_duplicates(), on='LAD22CD', how='inner')

# work out percentage of LAD for fund allocation
sums = merge.groupby('LAD22CD')['area_m2'].sum()
merge['percentage'] = merge.apply(lambda row: row['area_m2']/ sums[row['LAD22CD']] if sums[row['LAD22CD']] !=0 else 0,axis=1)
merge.drop(['area_m2'], axis=1, inplace=True)

# merge this with data
df0 = pd.merge(df0, merge.drop_duplicates(), on=['Segment_Name', 'Amount'], how='left')
df0.loc[df0['Area_Type']=='LEP', 'final_lad_code'] = df0.loc[df0['Area_Type']== 'LEP', 'LAD22CD']
df0.loc[df0['Area_Type']=='LEP', 'final_lad_name'] = df0.loc[df0['Area_Type']== 'LEP', 'LAD22NM']
df0.drop(['LAD22CD','LAD22NM',x + '22CD',x + '22NM'], axis=1, inplace=True)
df0['Derived_Amount'] = df0['Amount']*df0['percentage']
df0['Derived_Amount'] = df0['Derived_Amount'].round(2)
df0.drop(['percentage'],axis=1, inplace=True)

# BUA
x = 'BUA'

df3 = pd.read_csv('D:/Users/daniel.godden/Data/lookup_data/BUA_LAD.csv')
df2 = lookup_dict[x]
merged = pd.merge(df2, df3.drop_duplicates(), on=['LAD22CD', 'LAD22NM', 'BUA22CD', 'BUA22NM'], how='inner')
merge = df0.loc[df0['Area_Type']== x]
merge = merge[['Segment_Name','Local_Authority_Code', 'Amount']]
merge['LAD22CD'] = merge['Local_Authority_Code']
merge.drop(['Local_Authority_Code'], axis=1, inplace=True)
merge = pd.merge(merged, merge.drop_duplicates(), on='LAD22CD', how='inner')

# work out percentage of LAD for fund allocation
sums = merge.groupby('LAD22CD')['area_m2'].sum()
merge['percentage'] = merge.apply(lambda row: row['area_m2']/ sums[row['LAD22CD']] if sums[row['LAD22CD']] !=0 else 0,axis=1)
merge.drop(['area_m2'], axis=1, inplace=True)
merge.drop(['BUA22CD', 'BUA22NM'], axis=1, inplace=True)

# merge this with data
df0 = pd.merge(df0, merge.drop_duplicates(), on=['Segment_Name', 'Amount'], how='left')
df0.loc[df0['Area_Type']=='BUA', 'final_lad_code'] = df0.loc[df0['Area_Type']== 'BUA', 'LAD22CD']
df0.loc[df0['Area_Type']=='BUA', 'final_lad_name'] = df0.loc[df0['Area_Type']== 'BUA', 'LAD22NM']
df0.drop(['LAD22CD','LAD22NM'], axis=1, inplace=True)
df0.loc[df0['Area_Type']==x, 'Derived_Amount'] = df0.loc[df0['Area_Type']== x, 'Amount']*df0.loc[df0['Area_Type']==x, 'percentage']
df0['Derived_Amount'] = df0['Derived_Amount'].round(2)
df0.drop(['percentage'],axis=1, inplace=True)

# CAUTH (MCA/CA)
x ='CAUTH'

df2 = lookup_dict[x]
merge = df0.loc[df0['Area_Type']== x]
merge = merge[['Segment_Name','Local_Authority_Code', 'Amount']]
merge['LAD22CD'] = merge['Local_Authority_Code']
merge.drop(['Local_Authority_Code'], axis=1, inplace=True)
merge = pd.merge(df2, merge.drop_duplicates(), on='LAD22CD', how='inner')

# work out percentage of LAD for fund allocation
sums = merge.groupby('LAD22CD')['area_m2'].sum()
merge['percentage'] = merge.apply(lambda row: row['area_m2']/ sums[row['LAD22CD']] if sums[row['LAD22CD']] !=0 else 0,axis=1)
merge.drop(['area_m2'], axis=1, inplace=True)

# merge this with data
df0 = pd.merge(df0, merge.drop_duplicates(), on=['Segment_Name', 'Amount'], how='left')
df0.loc[df0['Area_Type']=='CAUTH', 'final_lad_code'] = df0.loc[df0['Area_Type']== 'CAUTH', 'LAD22CD']
df0.loc[df0['Area_Type']=='CAUTH', 'final_lad_name'] = df0.loc[df0['Area_Type']== 'CAUTH', 'LAD22NM']
df0.drop(['LAD22CD','LAD22NM',x + '22CD',x + '22NM'], axis=1, inplace=True)
df0.loc[df0['Area_Type']==x, 'Derived_Amount'] = df0.loc[df0['Area_Type']== x, 'Amount']*df0.loc[df0['Area_Type']==x, 'percentage']
df0['Derived_Amount'] = df0['Derived_Amount'].round(2)
df0.drop(['percentage'],axis=1, inplace=True)

# freeport
x ='freeport'

df2 = lookup_dict[x]
merge = df0.loc[df0['Area_Type']== x]
merge = merge[['Segment_Name','Local_Authority_Code', 'Amount']]
merge['LAD22CD'] = merge['Local_Authority_Code']
merge.drop(['Local_Authority_Code'], axis=1, inplace=True)
merge = pd.merge(df2, merge.drop_duplicates(), on='LAD22CD', how='inner')

# work out percentage of LAD for fund allocation
sums = merge.groupby('LAD22CD')['area_m2'].sum()
merge['percentage'] = merge.apply(lambda row: row['area_m2']/ sums[row['LAD22CD']] if sums[row['LAD22CD']] !=0 else 0,axis=1)
merge.drop(['area_m2'], axis=1, inplace=True)

# merge this with data
df0 = pd.merge(df0, merge.drop_duplicates(), on=['Segment_Name', 'Amount'], how='left')
df0.loc[df0['Area_Type']=='freeport', 'final_lad_code'] = df0.loc[df0['Area_Type']== 'freeport', 'LAD22CD']
df0.loc[df0['Area_Type']=='freeport', 'final_lad_name'] = df0.loc[df0['Area_Type']== 'freeport', 'LAD22NM']
df0.drop(['LAD22CD','LAD22NM'], axis=1, inplace=True)
df0.loc[df0['Area_Type']==x, 'Derived_Amount'] = df0.loc[df0['Area_Type']== x, 'Amount']*df0.loc[df0['Area_Type']==x, 'percentage']
df0['Derived_Amount'] = df0['Derived_Amount'].round(2)
df0.drop(['percentage'],axis=1, inplace=True)


In [15]:
df0.head()

,Coverage,Region,Local_Authority_Name,Local_Authority_Code,Town_or_City_Name,Postcode,Where,Who,Amount,Control_Budget,...,GEOGNM_derived,GEOGCD_TYPE_derived,GEOGCD_original,GEOGNM_original,GEOGCD_TYPE_original,final_lad_code,final_lad_name,Derived_Amount,Area_Type,Freeport
0,England,West Midlands,"Herefordshire, County of",E06000019,HEREFORD,HR4 0LE,NaN,NaN,33162.98,DEL,...,"Herefordshire, County of",LAD22CD,E06000019,"Herefordshire, County of",LAD22CD,E06000019,"Herefordshire, County of",753.70,LEP,NaN
1,England,West Midlands,"Herefordshire, County of",E06000019,HEREFORD,HR4 0LE,NaN,NaN,290298.52,DEL,...,"Herefordshire, County of",LAD22CD,E06000019,"Herefordshire, County of",LAD22CD,E06000019,"Herefordshire, County of",6597.69,LEP,NaN
2,England,West Midlands,"Herefordshire, County of",E06000019,HEREFORD,HR4 0LE,NaN,NaN,11171.58,DEL,...,"Herefordshire, County of",LAD22CD,E06000019,"Herefordshire, County of",LAD22CD,E06000019,"Herefordshire, County of",253.90,LEP,NaN
3,England,West Midlands,"Herefordshire, County of",E06000019,HEREFORD,HR4 0LE,NaN,NaN,35279.36,DEL,...,"Herefordshire, County of",LAD22CD,E06000019,"Herefordshire, County of",LAD22CD,E06000019,"Herefordshire, County of",801.80,LEP,NaN
4,England,West Midlands,"Herefordshire, County of",E06000019,HEREFORD,HR4 0XH,NaN,NaN,124306.00,DEL,...,"Herefordshire, County of",LAD22CD,E06000019,"Herefordshire, County of",LAD22CD,NaN,NaN,NaN,NaN,NaN


If the knowledge_fund didn't pick anything up:

In [16]:
# if GEOGCD_TYPE_derived is 'LAD' level and Area_Type is 'NaN, populate new columns
df0.loc[(df0['GEOGCD_TYPE_derived']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'Derived_Amount'] = df0.loc[(df0['GEOGCD_TYPE_derived']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'Amount']
df0.loc[(df0['GEOGCD_TYPE_derived']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'final_lad_code'] = df0.loc[(df0['GEOGCD_TYPE_derived']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'Local_Authority_Code']
df0.loc[(df0['GEOGCD_TYPE_derived']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'final_lad_name'] = df0.loc[(df0['GEOGCD_TYPE_derived']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'Local_Authority_Name']

# if GEOGCD_TYPE_original is 'LAD' level and Area_Type is 'NaN, populate new columns
df0.loc[(df0['GEOGCD_TYPE_original']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'Derived_Amount'] = df0.loc[(df0['GEOGCD_TYPE_original']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'Amount']
df0.loc[(df0['GEOGCD_TYPE_original']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'final_lad_code'] = df0.loc[(df0['GEOGCD_TYPE_original']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'Local_Authority_Code']
df0.loc[(df0['GEOGCD_TYPE_original']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'final_lad_name'] = df0.loc[(df0['GEOGCD_TYPE_original']=='LAD22CD') & (df0['Area_Type']=='NaN'), 'Local_Authority_Name']